# data-utils tests
---

Experimenting with data-utils methods to check if everything is working properly.

## Importing the necessary packages

In [ ]:
import dask.dataframe as dd                # Dask to handle big data in dataframes
import pandas as pd                        # Pandas to load the data initially
from dask.distributed import Client        # Dask scheduler
import os                                  # os handles directory/workspace changes
import numpy as np                         # NumPy to handle numeric and NaN operations
import torch                               # PyTorch to create and apply deep learning models
import data_utils as du                    # Data science and machine learning relevant methods

In [ ]:
du.random_seed

In [ ]:
du.set_random_seed(42)

In [ ]:
du.random_seed

In [ ]:
# Debugging packages
import pixiedust                           # Debugging in Jupyter Notebook cells

In [ ]:
# Change to parent directory (presumably "Documents")
os.chdir('../..')

# Path to the CSV dataset files
data_path = 'Datasets/Others/digidb/'

In [ ]:
# Set up local cluster
client = Client()
client

In [ ]:
client.run(os.getcwd)

## Loading data

Opting on testing with a [Digimon dataset from Kaggle](https://www.kaggle.com/rtatman/digidb/data#), for nostalgia purposes 🙂

In [ ]:
df = dd.read_csv(f'{data_path}DigiDB_digimonlist.csv', dtype={'Digimon': str, 'Stage': str, 'Type': str, 'Attribute': str})
df.head()

Also creating artifical data for other tests:

In [ ]:
data_df = pd.DataFrame([[103, 0, '1;2'],
                        [103, 1, 3],
                        [104, 0, 4],
                        [105, 0, '3;1;2;4'],
                        [105, 1, '4;1;3']], columns=['id', 'ts', 'Var0'])
data_df

### Encoding categorical features

In [ ]:
df.dtypes

In [ ]:
df.Stage.dtype == 'object'

In [ ]:
feature = 'Attribute'

In [ ]:
# Clean the column's string values to have the same, standard format
tmp_df = du.data_processing.clean_naming(df, feature)
tmp_df.head()

In [ ]:
# Get the unique values of the cateforical feature
unique_values = df[feature].unique()
if 'dask' in str(type(df)):
    # Make sure that the unique values are computed, in case we're using Dask
    unique_values = unique_values.compute()
# Enumerate the unique values in the categorical feature and put them in a dictionary
enum_dict = du.embedding.create_enum_dict(unique_values)

In [ ]:
enum_dict

In [ ]:
# Create a series from the enumerations of the original feature's categories
enum_series = df[feature].map(lambda x: du.utils.apply_dict_convertion(x, enum_dict, nan_value=0), meta=('x', int))
enum_series.head()

In [ ]:
df = df.compute()

In [ ]:
# [TODO] Sometimes in Dask, and only sometimes, for some reason this doesn't work, 
# as it considers that 'Attribute' column has the same unique values as 'Type' ¯\_(ツ)_/¯
# Try to find out what's wrong
feat_enum = dict()
for feature in df.columns:
    if df[feature].dtype == 'object':
        # Prepare for embedding, i.e. enumerate categories
        df[feature], feat_enum[feature] = du.embedding.enum_categorical_feature(df, feature)
df.head()

In [ ]:
feat_enum

### Create a dataset object:

In [ ]:
class Dataset(torch.utils.data.Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, df, label_name=None):
        # Counter that indicates in which column we're in when searching for the label column
        col_num = 0
        for col in df.columns:
            if 'label' in col or col == label_name:
                # Column number corresponding to the label
                self.label_column = col_num
                break
            col_num += 1
        # Convert the data into a NumPy array
        data = df.to_numpy()
        # Column numbers corresponding to the features
        self.features_columns = list(range(self.label_column)) + list(range(self.label_column + 1, data.shape[1]))
        # Features
        self.X = data[:, self.features_columns]
        # Labels
        self.y = data[:, self.label_column]

    def __getitem__(self, item):
        x = self.X[item]
        y = self.y[item]
        return x, y

    def __len__(self):
        return len(self.X)

In [ ]:
dataset = Dataset(df, 'Attribute')

In [ ]:
dataset.__len__()

In [ ]:
dataset.X

In [ ]:
dataset.y

In [ ]:
dataset.__getitem__(0)

## Testing data utils methods and their reproducibility

### Separating in different sets

In [ ]:
train_dataloader, val_dataloader, test_dataloader = du.machine_learning.create_train_sets(dataset, get_indeces=False)

In [ ]:
val_dataloader

In [ ]:
next(iter(val_dataloader))

In [ ]:
for x, y in val_dataloader:
    print(y)

Everything seems to be working great! (at least with the NumPy's random seed; still need to confirm on PyTorch)

### Converting string encodings to numeric

In [ ]:
data_df_num = du.embedding.string_encod_to_numeric(data_df)
data_df_num

In [ ]:
data_df_num.dtypes